<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/sglogo.png width=50 align="left">

# TOTP MFA
<img src="images/vault-totp1.png" width=300>  

MFA works only for tokens that have identity information on them
Tokens created by logging in using auth methods will have the associated identity information

**Prerequisites:** 
You need to have Enterprise Vault up and running

`VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 evault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional`

In [2]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [3]:
vault login root

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       1yDVFzpd0DllvRwkhbX1NPtT
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [4]:
vault status; vault version

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.7.0+ent
Storage Type    inmem_transactional_ha
Cluster Name    vault-cluster-1141985c
Cluster ID      4855d8f8-cc99-b3dd-13f5-c54ae795613a
HA Enabled      true
HA Cluster      https://0.0.0.0:8201
HA Mode         active
Active Since    2021-03-27T15:15:04.519803Z
Last WAL        35
Vault v1.7.0 (4e222b85c40a810b74400ee3c54449479e32bb9f)


### Create static secrets

In [5]:
vault secrets enable -version=1 -path=secret1 kv

Success! Enabled the kv secrets engine at: secret1/


In [6]:
vault kv put secret1/web01 ip="192.168.1.191" username="arthur" password="passw0rd123"
vault kv put secret1/web02 ip="192.168.1.192" username="raymond" password="passw0rd123"

Success! Data written to: secret1/web01
Success! Data written to: secret1/web02


---
## Configure TOTP MFA

In [7]:
vault write sys/mfa/method/totp/my_totp \
    issuer=Vault \
    period=30 \
    key_size=30 \
    algorithm=SHA256 \
    digits=6

Success! Data written to: sys/mfa/method/totp/my_totp


In [8]:
vault policy write totp-policy -<<EOF
path "secret1/*" {
  capabilities = ["read"]
  mfa_methods  = ["my_totp"]
}
EOF

Success! Uploaded policy: totp-policy


**Note:** the policy above allows reading of `secret1` but it also requires MFA `my_totp`

### Use Userpass authentication

In [9]:
vault auth enable userpass

Success! Enabled userpass auth method at: userpass/


In [10]:
vault write auth/userpass/users/tiobagio \
    password=passw0rd \
    policies=totp-policy

Success! Data written to: auth/userpass/users/tiobagio


In [11]:
vault write auth/userpass/login/tiobagio \
    password=passw0rd

Key                    Value
---                    -----
token                  s.eiaWMTXRNJf5O5NUN1aS0Ryp
token_accessor         J1FjbmEMZQD7pypWuCJPTuwd
token_duration         768h
token_renewable        true
token_policies         ["default" "totp-policy"]
identity_policies      []
policies               ["default" "totp-policy"]
token_meta_username    tiobagio


**Note:** the CLI is not authenticated with the newly created token yet, we **did not** call `vault login`, instead we used the login API to simply return a token

In [12]:
VAULT_TOKEN=root vault token lookup s.eiaWMTXRNJf5O5NUN1aS0Ryp

Key                 Value
---                 -----
accessor            J1FjbmEMZQD7pypWuCJPTuwd
creation_time       1616858164
creation_ttl        768h
display_name        userpass-tiobagio
entity_id           7777ca15-2f47-68d0-8953-687656c95783
expire_time         2021-04-28T23:16:04.233537+08:00
explicit_max_ttl    0s
id                  s.eiaWMTXRNJf5O5NUN1aS0Ryp
issue_time          2021-03-27T23:16:04.233543+08:00
meta                map[username:tiobagio]
num_uses            0
orphan              true
path                auth/userpass/login/tiobagio
policies            [default totp-policy]
renewable           true
ttl                 767h59m51s
type                service


### MFA works only for tokens that have identity information on them
Tokens created by logging in using auth methods will have the associated identity information

In [13]:
#use entity_id from the previous output
vault write sys/mfa/method/totp/my_totp/admin-generate \
    entity_id=7777ca15-2f47-68d0-8953-687656c95783

Key        Value
---        -----
barcode    iVBORw0KGgoAAAANSUhEUgAAAMgAAADIEAAAAADYoy0BAAAG00lEQVR4nOydwY7sOAhFp0fv/3/5zaI2biHQARzNTemeVStxbFdfgQBTqT9///5jhPj3/96A+Y0FEcOCiGFBxLAgYlgQMSyIGBZEDAsihgURw4KIYUHEsCBiWBAxLIgYFkQMCyKGBRHDgojxhw78+aEj96f0n7U+85zrnlfi3TiG7CebPxLneeJ/YgsRw4KIgV3WB27+55XzqXilniGuG/+Oc8YV4/j6Snw223n3f1JjCxHDgojRdFkfstgmjqkNvJ4nRlN1lDWLeWqnWu/2hPxPCLYQMSyIGCOXxSFJ3DkycyD1zPWKdULHnWo95y1sIWJYEDEedlmZQ8gSrtNxxRmyxLOmG//wtPEJbCFiWBAxRi6Lm22WZNUz1+X3OHNcMZstG1PPSbjlymwhYlgQMZouq1tMJpUoknCRRK+bJO7nPK/cwhYihgURA7usW60L2d/ZeV89Tz2eV582Tu8uthAxLIgYP9Ts6kL6SffUr16r3k+9h25DBY+1+B662ELEsCBiNF3WCWkVyEbGeeoSd7d9NNI9N+QjSbOrW0lfiwURo5kY1gldZryzTqfufroxEtnJuWd+XrlJHm0hYlgQMXCU9esh3NPe7cviFSdSBicJ3WxMHeltTiFtIWJYEDEWJ4Z11FE3NnSL4bXrmyWVdTqZ9Ylln2KTDJ7YQsSwIGKMvmPYLUrzSldchVzp1tDqVoeYzBJHt69ifbCFiGFBxGgmhrOvwPBEjKRgJ7Oy+a109Ymv89hCxLAgYjRfPsO702eznXezOesjgDgmm5PsYZZOZp+UYAsRw4KIsWhy+EBaHXhal61FziWzvXUjpXiXt3Nke+bYQsSwIGJc+o4hj7tI2kiiIFIB65bx45i4OnFEm/jTFiKGBRFj1JdFjP28QuaM85AZSLm

**Note**: the above barcode is a .png file that looks like the picture below.  Use
`echo "<insert barcode value here>" | base64 --decode > /tmp/barcode.png` 

<img src="images/barcode.png" width=100>

### Use Google Authenticator
Open Google Authenticator, click + (plus sign) and press "Scan barcode".   
<img src="images/authenticator1.png" width=100>
<img src="images/authenticator2.png" width=100>

### Demo - Read the secret

In [14]:
unset VAULT_TOKEN;vault login s.eiaWMTXRNJf5O5NUN1aS0Ryp

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                    Value
---                    -----
token                  s.eiaWMTXRNJf5O5NUN1aS0Ryp
token_accessor         J1FjbmEMZQD7pypWuCJPTuwd
token_duration         767h56m56s
token_renewable        true
token_policies         ["default" "totp-policy"]
identity_policies      []
policies               ["default" "totp-policy"]
token_meta_username    tiobagio


In [15]:
vault print token; vault kv get secret1/web01

s.eiaWMTXRNJf5O5NUN1aS0Ryp
Error reading secret1/web01: Error making API request.

URL: GET http://127.0.0.1:8200/v1/secret1/web01
Code: 403. Errors:

* 1 error occurred:
	* permission denied




**Note**: OTP is required!!

In [16]:
vault print token;  vault read -mfa my_totp:443937 secret1/web01

s.eiaWMTXRNJf5O5NUN1aS0Ryp
Error reading secret1/web01: Error making API request.

URL: GET http://127.0.0.1:8200/v1/secret1/web01
Code: 403. Errors:

* 1 error occurred:
	* permission denied




**Note**: Use Google Authenticator to get 6 digits number. Using wrong OTP results in `permission denied`

In [17]:
vault print token;  vault read -mfa my_totp:238670 secret1/web01

s.eiaWMTXRNJf5O5NUN1aS0Ryp
Key                 Value
---                 -----
refresh_interval    768h
ip                  192.168.1.191
password            passw0rd123
username            arthur


---
### TOTP Secret Engine

In [21]:
export VAULT_TOKEN=root; vault secrets enable totp

Success! Enabled the totp secrets engine at: totp/


### Use url from above

In [22]:
VAULT_TOKEN=root; 
vault write totp/keys/my-key \
    url="otpauth://totp/Vault:7777ca15-2f47-68d0-8953-687656c95783?algorithm=SHA256&digits=6&issuer=Vault&period=30&secret=DAHU3AFEUEIYN3NZWHCO2NLTXUYDDJWM7AN36AWMCLMVT6ZB"

Success! Data written to: totp/keys/my-key


In [27]:
export VAULT_TOKEN=root; vault read totp/code/my-key

Key     Value
---     -----
code    813039


In [28]:
VAULT_TOKEN=s.eiaWMTXRNJf5O5NUN1aS0Ryp;
vault read secret1/web01

Error reading secret1/web01: Error making API request.

URL: GET http://127.0.0.1:8200/v1/secret1/web01
Code: 403. Errors:

* 1 error occurred:
	* permission denied




**Note**: OTP is required!!

In [29]:
VAULT_TOKEN=s.eiaWMTXRNJf5O5NUN1aS0Ryp;
vault read -mfa my_totp:813039 secret1/web01

Key                 Value
---                 -----
refresh_interval    768h
ip                  192.168.1.191
password            passw0rd123
username            arthur


In [30]:
VAULT_TOKEN=root
vault write totp/code/my-key code=813039

Key      Value
---      -----
valid    true


In [31]:
VAULT_TOKEN=root
vault write totp/code/my-key code=813030

Key      Value
---      -----
valid    false


&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">